In [1]:
import numpy as np

In [2]:
class Item():
    
    def __init__(self, value, weight):
        self.v = value
        self.w = weight
        self.density = value/weight
    
    def get_density(self):
        return self.density
    
    def __str__(self):
        return "value: "+str(self.v)+" weight: "+str(self.w)+" density: "+str(self.get_density())

In [3]:
n = 10
np.random.seed(2376)
I = [Item(np.random.randint(low=1, high=100), np.random.randint(low=2, high=20)) for _ in range(n)]
V = []
maxWeight = 30

In [ ]:
I[0].get_density()

### 1. Полный перебо

In [18]:
def maxVal(items, maxWeight):
    if items == [] or maxWeight == 0:
        # end of tree
        result = (0, ())
    elif items[0].w > maxWeight:
        # so large
        result = maxVal(items[1:], maxWeight)
    else:
        # take or don't
        nextItem = items[0]
        withVal, withToTake = maxVal(items[1:], maxWeight - nextItem.w)
        withVal += nextItem.v
        withoutVal, withoutToTake = maxVal(items[1:], maxWeight)
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    return result

In [23]:
V = fastMaxVal(I, maxWeight)
print(V[0])
for it in V[1]:
    print(it)

247
value: 46 weight: 9 density: 5.111111111111111
value: 49 weight: 2 density: 24.5
value: 18 weight: 2 density: 9.0
value: 64 weight: 8 density: 8.0
value: 70 weight: 4 density: 17.5


### 2. Динамичсекое программирование

In [16]:
def fastMaxVal(items, maxWeight, memo={}):
    if items == [] or maxWeight == 0:
        # end of tree
        result = (0, ())
    elif items[0].w > maxWeight:
        # so large
        result = fastMaxVal(items[1:], maxWeight)
    else:
        # take or don't
        nextItem = items[0]
        withVal, withToTake = fastMaxVal(items[1:], maxWeight - nextItem.w)
        withVal += nextItem.v
        withoutVal, withoutToTake = fastMaxVal(items[1:], maxWeight)
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    return result

In [17]:
%%time
fastMaxVal(I, maxWeight)

CPU times: total: 0 ns
Wall time: 995 µs


(247,
 (<__main__.Item at 0x2a0cb0eddc0>,
  <__main__.Item at 0x2a0c9b665b0>))

### 3. Жадный алгоритм

In [4]:
def greedy(items, maxWeight, keyFunction):
    itemsCopy = sorted(items, key = keyFunction, reverse = True)
    result = []
    totalValue, totalWeight = 0.0, 0.0
    for i in range(len(itemsCopy)):
        if (totalWeight+itemsCopy[i].w) <= maxWeight:
            result.append(itemsCopy[i])
            totalWeight += itemsCopy[i].w
            totalValue += itemsCopy[i].v
    return (result, totalValue)

def testGreedy(items, constraint, keyFunction):
    taken, val = greedy(items, constraint, keyFunction)
    print('Total value of items taken =', val)
    for item in taken:
        print(' ', item)

In [5]:
V = testGreedy(I, maxWeight, Item.get_density)

Total value of items taken = 247.0
  value: 49 weight: 2 density: 24.5
  value: 70 weight: 4 density: 17.5
  value: 18 weight: 2 density: 9.0
  value: 64 weight: 8 density: 8.0
  value: 46 weight: 9 density: 5.111111111111111


### 4. Случайный перебор